In [17]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [18]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Kaitangata,NZ,2020-09-06 20:44:07,-46.28,169.85,48.00,72,34,8.99
1,1,Carnarvon,AU,2020-09-06 20:46:38,-24.87,113.63,68.00,94,75,8.05
2,2,Baykit,RU,2020-09-06 20:46:38,61.67,96.37,41.45,82,28,1.45
3,3,Ancud,CL,2020-09-06 20:46:38,-41.87,-73.82,55.99,74,0,1.99
4,4,Tiznit,MA,2020-09-06 20:46:39,29.58,-9.50,83.71,36,0,1.81


In [19]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [20]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
# heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

NameError: name 'temps' is not defined

In [21]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
# Heatmap of percent wind
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [24]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
4,4,Tiznit,MA,2020-09-06 20:46:39,29.58,-9.50,83.71,36,0,1.81
5,5,Billings,US,2020-09-06 20:46:11,45.78,-108.50,81.00,24,1,6.93
6,6,Puerto Ayora,EC,2020-09-06 20:45:13,-0.74,-90.35,77.00,53,20,21.92
7,7,Kapaa,US,2020-09-06 20:43:26,22.08,-159.32,84.20,66,20,16.11
13,13,Langsa,ID,2020-09-06 20:46:40,4.47,97.97,75.13,90,100,2.71
14,14,Fukuma,JP,2020-09-06 20:46:40,33.77,130.47,89.01,78,75,51.67
20,20,Bac Lieu,VN,2020-09-06 20:46:41,9.29,105.72,79.88,82,39,6.82
30,30,Casian,PH,2020-09-06 20:46:44,11.04,119.71,84.02,70,93,5.99
40,40,Atuona,PF,2020-09-06 20:46:46,-9.80,-139.03,79.03,70,75,18.75
47,47,Hobyo,SO,2020-09-06 20:46:47,5.35,48.53,76.35,82,0,21.88


In [25]:
preferred_cities_df.count()

City_ID       184
City          184
Country       184
Date          184
Lat           184
Lng           184
Max Temp      184
Humidity      184
Cloudiness    184
Wind Speed    184
dtype: int64

In [26]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Tiznit,MA,83.71,29.58,-9.50,
5,Billings,US,81.00,45.78,-108.50,
6,Puerto Ayora,EC,77.00,-0.74,-90.35,
7,Kapaa,US,84.20,22.08,-159.32,
13,Langsa,ID,75.13,4.47,97.97,
14,Fukuma,JP,89.01,33.77,130.47,
20,Bac Lieu,VN,79.88,9.29,105.72,
30,Casian,PH,84.02,11.04,119.71,
40,Atuona,PF,79.03,-9.80,-139.03,
47,Hobyo,SO,76.35,5.35,48.53,


In [28]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

# Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try: 
      hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.


KeyboardInterrupt: 